In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'files/Cal5DoF05222023-post FDPR V3.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.957073   0.213787   0.797286  0.026155   0.003663
PR2  -108.548109 -46.839383  23.430574 -4.906209  10.653646
PR3   112.267199 -46.530542  23.278306 -4.873173 -10.626984
PR4   104.116729  58.616016  31.900208  5.293205 -10.348396
PR5  -100.390912  58.423929  31.865299  5.273991  10.373671 

Provided 5DOF encoder values: 
                X.1        Y.1        Z.1      Rx.1       Ry.1
Name.1                                                       
PR1       1.933524   0.250412   0.783184  0.026885   0.005745
PR2    -108.452857 -46.898136  23.320940 -4.906167  10.652669
PR3     112.219072 -46.231812  23.378512 -4.871664 -10.626735
PR4     103.923673  58.840103  32.096062  5.298097 -10.345960
PR5    -100.448696  58.292823  31.861952  5.273415  10.374527


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [5]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,Rz,uvec_X,uvec_Y,uvec_Z
Name,,,,,,,,,,
PR1,1.957073,0.213787,0.797286,0.026155,0.003663,crimson,NaN,0.082611,248.260690,1268.049080
PR2,-108.548109,-46.839383,23.430574,-4.906209,10.653646,crimson,NaN,238.876843,350.225013,1220.599091
PR3,112.267199,-46.530542,23.278306,-4.873173,-10.626984,crimson,NaN,-238.285909,349.551738,1220.907554
PR4,104.116729,58.616016,31.900208,5.293205,-10.348396,crimson,NaN,-232.108195,128.680371,1264.574630
PR5,-100.390912,58.423929,31.865299,5.273991,10.373671,crimson,NaN,232.668903,129.094033,1264.429420
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN,NaN
sMPA,2.04,248.465,1268.826,-24.297100,0.007400,purple,0.0746,0.000129,0.411468,0.911424


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [6]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('files/FDPR shifts - sandbox.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                                ...                           
PR1                   -334               -343  307.5  ...    -51.5     -161     -515
PR3                   -377               -389  300.5  ...    -44.5     -295     -445
PR4                   -335               -392  309.2  ...    -53.2     -219     -532
PR5                   -368               -430  306.4  ...    -50.4     -145     -504
PR2                   -310               -319  292.9  ...    -36.9     -172     -369

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [7]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

## Scratch

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 5DOF optimization)

In [8]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals (in sMPA frame) baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.160351  0.535467  0.124598
PR2  0.230493  0.422974  0.149398
PR3  0.222683  0.511152  0.184264
PR4  0.147891  0.524164  0.164599
PR5  0.221773  0.502482  0.213309 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.1842  0.2993  0.2059 -0.0092  0.0006] 

Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.036    0.033   -0.041
PR2    0.034   -0.071    0.002
PR3    0.026    0.016    0.032
PR4   -0.049    0.022   -0.023
PR5    0.025    0.001    0.030 

endpt_X    0.035116
endpt_Y    0.037132
endpt_Z    0.029009
dtype: float64


### 1.2 (alternate) Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [9]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.160351  0.535467  0.124598
PR2  0.230493  0.422974  0.149398
PR3  0.222683  0.511152  0.184264
PR4  0.147891  0.524164  0.164599
PR5  0.221773  0.502482  0.213309 

Best-fit 6DoF deltas to apply to real-space poses: 
 [ 0.1844  0.2995  0.2059 -0.0092 -0.0009 -0.0075] 

Residuals after incorporating the best-fit 6DoF shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.037    0.033   -0.041
PR2    0.019   -0.042   -0.004
PR3    0.012   -0.014    0.038
PR4   -0.034   -0.008   -0.017
PR5    0.040    0.031    0.025 

endpt_X    0.030406
endpt_Y    0.028366
endpt_Z    0.028532
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [10]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
               X          Y          Z        Rx         Ry
PR1    2.117471   0.553514   0.986823  0.017713   0.006319
PR2  -108.27037 -46.587335   23.51987 -4.913985   10.65387
PR3  112.402472 -45.927157  23.581703 -4.882214 -10.625538
PR4  104.107686  59.139151  32.302725  5.287544 -10.346076
PR5 -100.265531  58.597562   32.06467  5.265611  10.374415 

Baseline pose encoders (displayed again here just for reference): 
                X.1        Y.1        Z.1      Rx.1       Ry.1
Name.1                                                       
PR1       1.933524   0.250412   0.783184  0.026885   0.005745
PR2    -108.452857 -46.898136  23.320940 -4.906167  10.652669
PR3     112.219072 -46.231812  23.378512 -4.871664 -10.626735
PR4     103.923673  58.840103  32.096062  5.298097 -10.345960
PR5    -100.448696  58.292823  31.861952  5.273415  10.374527


# 2 Write the new poses to an Excel file in the current directory

In [11]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from FDPR update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)

**Writing to Excel complete**
